# NIQE and Ma et al. metrics study on satellite imagery

In this notebook we will investigate how the tile size of satellite imagery affects [NIQE](https://ieeexplore.ieee.org/document/6353522) and [Ma et al.](https://www.sciencedirect.com/science/article/pii/S107731421630203X) (short form *Ma*) scores.

We will generate random tile samples of varying size from two satellite sensors, WorldView-2 and GeoEye-1, and use an integration with MATLAB through the [MATLAB Engine API for Python](https://se.mathworks.com/help/matlab/matlab-engine-for-python.html) to evaluate *NIQE* and *Ma et al.* scores

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

from modules.matlab_metrics import *
from modules.image_utils import *
from modules.helpers import *
from modules.tile_generator import *

In [ ]:
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)

SENSORS = ['WV02', 'GE01']
AREAS = ['La_Spezia', 'Toulon']

meta = subset_by_areas_sensor(meta, areas=AREAS, sensors=SENSORS)

# Path to location where individual satellite images are located
DATA_PATH = 'data/toulon-laspezia/' 
DATA_PATH_TILES = 'data/niqe-ma-study'

N_IMAGES = count_images(meta)
N_TILES = 100

SR_FACTOR = 4
MS_SIZES = [24, 32, 48, 64, 72, 96, 120, 128, 144, 160, 168, 192, 224, 256]
PAN_SIZES = [size*SR_FACTOR for size in MS_SIZES]

meta = allocate_tiles(meta, by_partition=False, n_tiles_total=N_TILES)}

In [ ]:
tile_path = 'data/toulon-laspezia-tiles-large/WV02/train/WV02_La_Spezia_2012_01_10_011650582010_0/pan/00006.tif'
tile_path = 'data/toulon-laspezia-tiles-noseaclouds/WV02/train/WV02_La_Spezia_2012_05_17_011650585010_0/pan/00006.tif'
test_img = 'data/PIRM_dataset/Validation/Original/2.png'
#test_img = 'data/test.png'
n=2
mu = 370.93775769104

In [ ]:
img = geotiff_to_ndarray(tile_path)
img[0,0,0] = 0
img[1,1,0] = 2047
mu = np.mean(img)
print(img.dtype, np.min(img), np.max(img), np.mean(img))
img_sc = input_scaler(img, radius=1.0, output_dtype='float32', uint_bit_depth=11,
                      mean_correction=None, mean=0, print_ranges=True)
print(img_sc.dtype, np.min(img_sc), np.max(img_sc), np.mean(img_sc), 'range:', np.max(img_sc)-np.min(img_sc))
#img_unsc = output_scaler(img_sc, radius=1.0, output_dtype='uint16', uint_bit_depth=11,
#                         mean_correction=True, mean=mu)
#print(img_unsc.dtype, np.min(img_unsc), np.max(img_unsc), np.mean(img_unsc))
plt.imshow(img_sc, cmap='gray')


In [ ]:
plt.hist(np.ndarray.flatten(img))

In [ ]:
plt.hist(np.ndarray.flatten(img_sc))

In [ ]:
imgs = np.stack([img_sc]*n)
imgs.shape
#imgs = imgs.astype('uint8')
#imgs.shape
#imgs.dtype


In [ ]:
matlab_engine = MatLabEngine('modules/matlab', ma=True, niqe=True, 
                             input_range=(-1,1), output_dtype='uint16', output_bit_depth=11,
                             scale_mean=mu, stretch=False, shave_width=4)

In [ ]:
tic = time.perf_counter()
niqes = matlab_engine.matlab_niqe_metric(imgs)
toc = time.perf_counter()
print('niqe', niqes)
print(toc-tic, 's')
print((toc-tic)/n, 's per img')

In [ ]:
tic = time.perf_counter()
niqes = matlab_engine.skvideo_niqe_metric(imgs)
toc = time.perf_counter()
print('niqe', niqes)
print(toc-tic, 's')
print((toc-tic)/n, 's per img')

In [ ]:
tic = time.perf_counter()
ma = matlab_engine.matlab_ma_metric(imgs)
toc = time.perf_counter()
print('ma', ma)
print(toc-tic, 's')
print((toc-tic)/n, 's per img')

In [ ]:
ma = tf.constant([0,1,2,3,4,5,6,7,8,9], dtype=tf.float32)
tens = 10 * tf.ones_like(ma)
tens